In [ ]:
# TODO Bikin gen yang isinya random (99,99%)
# TODO Fitness/Cost Function (95%)
# TODO Generate Population (100%)
# TODO Selection (100%)
# TODO Crossover (100%)
# TODO Mutation (70%)
# TODO reassign population (eliminate weak gen) (100%)
# TODO Main Looping and Termination
# TODO Logging
# TODO Finalization
# TODO Develop Website
# TODO Deploy Website

# di python evrything is reference, kalau mau cpy harus eksplisit

In [78]:
import json
import random
from copy import deepcopy


input_file = 'input.json'
output_file = 'classes/output3.json'

# Baca data json 
def load_data(path):
    with open(path, 'r') as read_file:
        data = json.load(read_file)

    return data

# Masukan data json ke variabel
test = load_data(input_file)

In [ ]:
# Test data, sudah masuk atau tidak
# print(test['Waktu']['Senin'])
print(test['List_Mapel'])

In [3]:
# Fungsi mengambil satu buah elemen random dari sebuah list
def randomizer(source: list):
    index = random.randrange(0, len(source))
    temp = source[index]
    return temp, index


In [36]:
# Test fungsi randomizer
keys_for_slicing = ["Mapel", "ID"]
rand_mapel = {key: randomizer(test['List_Mapel'])[0][key] for key in keys_for_slicing}
rand_guru, index_guru = randomizer(test['Guru'])
print(rand_mapel)
print(rand_guru)

{'Mapel': 'Ilmu Pengetahuan Sosial', 'ID': '2'}
Oman Mandala


In [97]:
# Generate sebuah gen/kromosom/individu, lalu return satu timetable
def generate_gen():
    space = {}

    for angkatan in test['Kelas']:
        kelas_temp = {}
        for kelas in test['Kelas'][angkatan]:
            hari_temp = {}
            for hari in test['Waktu']:
                alokasi_waktu = 0
                mapel_temp = {}

                list_mapel = deepcopy(test['List_Mapel'])
                list_guru = deepcopy(test['Guru'])
                for jam in test['Waktu'][hari]:
                    if (alokasi_waktu == 0):
                        temp_id = {}

                        temp_mapel, index_mapel = randomizer(list_mapel)
                        temp_guru, index_guru = randomizer(list_guru)
                        alokasi_waktu = int(temp_mapel['Alokasi_Jam'])
                        
                        list_mapel.pop(index_mapel)
                        list_guru.pop(index_guru)

                        temp_id['Mapel'] = temp_mapel['Mapel']
                        temp_id['ID'] = temp_mapel['ID']
                        temp_id['Guru'] = temp_guru
                        mapel_temp[temp_mapel['ID']] = temp_id
                        mapel_temp[temp_mapel['ID']]['Slot_waktu'] = []
                    
                    
                    mapel_temp[temp_mapel['ID']]['Slot_waktu'].append(jam)
                    alokasi_waktu = alokasi_waktu - 1
                hari_temp[hari] = mapel_temp
            kelas_temp[kelas] = hari_temp
        space[angkatan] = kelas_temp

    return space

In [40]:
# Test Fungsi Generate Gen
space = generate_gen()
# print(space)

In [ ]:
guru = {test['Guru'][i]: {'Mapel': [], 'Slot_waktu': []}
        for i in range(0, len(test['Guru']))}
for i in guru:
    guru[i]['Slot_waktu'] = {}
    for j in test['Waktu']:
        guru[i]['Slot_waktu'][j] = []
print(guru)

In [104]:
# Cost Function
def cost_function(gen):
    cost = 0
    guru = {test['Guru'][i]: {'Mapel':[],'Slot_waktu':[]} for i in range(0, len(test['Guru']))}
    for i in guru:
        guru[i]['Slot_waktu'] = {}
        for j in test['Waktu']:
            guru[i]['Slot_waktu'][j] = []
            

    for angkatan in gen:
        for kelas in gen[angkatan]:
            for hari in gen[angkatan][kelas]:
                
                mapel_sehari = {}
                mapel_sehari['Mapel'] = []
                mapel_sehari['Total_Jam'] = []
                # mapel_sehari['Mapel'].append(
                #     gen[angkatan][kelas][hari]['7:00 - 7:40']['ID'])
                # mapel_sehari['Total_Jam'].append(0)
                counter = 0

                counter_jam = 0
                for jam in gen[angkatan][kelas][hari]:
                    
                    # Koneksi guru ke mapel
                    guru[gen[angkatan][kelas][hari][jam]['Guru']]['Mapel'].append(gen[angkatan][kelas][hari][jam]['ID'])

                    # Masukkan slot waktu tiap guru
                    for slot in gen[angkatan][kelas][hari][jam]['Slot_waktu']:
                        guru[gen[angkatan][kelas][hari][jam]['Guru']]['Slot_waktu'][hari].append(slot)
                        
                    
                    # Mapel sehari ada apa aja ngitung total jam mapel
                    mapel_sehari['Mapel'].append(gen[angkatan][kelas][hari][jam]['ID'])
                    mapel_sehari['Total_Jam'].append(len(gen[angkatan][kelas][hari][jam]['Slot_waktu']))
                    
                # Jika ada mapel hanya satu jam pada satu hari (+1)
                if 1 in mapel_sehari['Total_Jam']:
                    cost += 1

                # Jika ada Mapel yang sama di satu hari (+1)
                # result = dict((i, mapel_sehari['Mapel'].count(i)) for i in mapel_sehari['Mapel'])
                # if len(result) < len(mapel_sehari['Mapel']):
                #     cost += 1

    # Jika ada Guru yang sama di slot waktu yang sama disetiap hari (irisan) (+1)
    for j in guru:
        for k in guru[j]['Slot_waktu']:
            result = dict((i, guru[j]['Slot_waktu'][k].count(i)) for i in guru[j]['Slot_waktu'][k])
            cost += len(guru[j]['Slot_waktu'][k]) - len(result)
            # print(j)
            # print(k)
            # print(guru[j]['Slot_waktu'][k])
            # print(result)
        
        # Jika Guru Memegang lebih dari 4 mapel (+1)
        if len(guru[j]['Mapel']) > 4:
            cost += 1
    # print(guru)
    # TODO (skip dulu tar belakangan)
    # Jika ada mapel yang sama di satu angkatan tapi beda guru (+1)
    return cost, guru


In [51]:
# Test Cost Function
space = generate_gen()

print(cost_function(space)[0])




329


In [6]:
# Generate Population
def generate_population(num_gen: int):
    population = []
    for i in range(0, num_gen):
        gen = {}
        # space = generate_gen()
        gen['Timetable'] = generate_gen()
        gen['Cost'] = cost_function(gen['Timetable'])[0]
        population.append(gen)
    return population

# print(max(hasil))
# print(population)


In [69]:
# Test generate_population
population = generate_population(4)
for individu in population:
    # print(individu['Timetable'])
    print(individu['Cost'])

387
399
388
423


In [107]:
# Selection
def selection(population):
    temp = {}

    population_temp = deepcopy(population)

    for i in range(0, len(population_temp)):
        temp[i] = population_temp[i]['Cost']

    index = min(temp, key=temp.get)
    # print(index)
    parent1 = deepcopy(population_temp[index])
    population_temp.pop(index)

    temp = {}
    for i in range(0, len(population_temp)):
        temp[i] = population_temp[i]['Cost']

    index = min(temp, key=temp.get)
    # print(index)
    parent2 = deepcopy(population_temp[index])
    population_temp.pop(index)

    temp = {}
    for i in range(0, len(population_temp)):
        temp[i] = population_temp[i]['Cost']

    index = min(temp, key=temp.get)
    # print(index)
    parent3 = deepcopy(population_temp[index])
    population_temp.pop(index)

    temp = {}
    for i in range(0, len(population_temp)):
        temp[i] = population_temp[i]['Cost']

    index = min(temp, key=temp.get)
    # print(index)
    parent4 = deepcopy(population_temp[index])
    population_temp.pop(index)
    population_temp.pop(index)

    temp = {}
    for i in range(0, len(population_temp)):
        temp[i] = population_temp[i]['Cost']

    index = min(temp, key=temp.get)
    # print(index)
    parent5 = deepcopy(population_temp[index])
    population_temp.pop(index)

    temp = {}
    for i in range(0, len(population_temp)):
        temp[i] = population_temp[i]['Cost']

    index = min(temp, key=temp.get)
    # print(index)
    parent6 = deepcopy(population_temp[index])
    # population_temp.pop(index)
    # population_temp.pop(index)

    # print(parent1['Cost'])
    # print(parent2['Cost'])

    # return 2 individu terbaik
    return parent1, parent2, parent3, parent4, parent5, parent6


In [94]:
# Test Selection Function (hasil return: 2 individu terbaik)
population = generate_population(4)
    
parent1, parent2 = selection(population)
# print(parent1['Cost'], parent2['Cost'])

for individu in population:
    # print(individu['Timetable'])
    print(individu['Cost'])



424
433
444
437


In [8]:
# Crossover Function
def crossover(parent1, parent2):
    
    # print(parent1['Cost'], parent2['Cost'])

    child1 = deepcopy(parent1)
    child2 = deepcopy(parent2)

    # print("Crossover test:")

    # print(child1['Cost'])
    # print(child2['Cost'])

    child1['Timetable']['VII'] = parent2['Timetable']['VII']
    child2['Timetable']['VII'] = parent1['Timetable']['VII']

    child1['Cost'] = cost_function(child1['Timetable'])[0]
    child2['Cost'] = cost_function(child2['Timetable'])[0]

    # print(child1['Cost'])
    # print(child2['Cost'])

    return child1, child2


In [9]:
# Test Crossover Function
population = generate_population(4)
parent1, parent2 = selection(population)
print(parent1['Cost'])
print(parent2['Cost'])

print("====")
child1, child2 = crossover(parent1, parent2)
print

print(child1['Cost'])
print(child2['Cost'])


415
425
====
416
411


In [10]:
# Reassign individu (eliminate first)

# delete defective gen
def eliminate(population):
    temp = {}
    for i in range(0, len(population)):
        temp[i] = population[i]['Cost']
    index = max(temp, key=temp.get)
    population.pop(index)

    temp = {}
    for i in range(0, len(population)):
        temp[i] = population[i]['Cost']
    index = max(temp, key=temp.get)
    population.pop(index)

# Assign children to population
def reassign(population, child1, child2):
    eliminate(population)
    gen1 = deepcopy(child1)
    gen2 = deepcopy(child2)

    population.append(gen1)
    population.append(gen2)


In [98]:
# Test Fungsi Reassign

population = generate_population(4)
for individu in population:
    # print(individu['Timetable'])
    print(individu['Cost'])
parent1, parent2 = selection(population)
child1, child2 = crossover(parent1, parent2)
print("========")
print(child1['Cost'])
print(child2['Cost'])

reassign(population, child1, child2)

print("========")
for individu in population:
    # print(individu['Timetable'])
    print(individu['Cost'])


438
436
428
426
423
445
428
426
423
445


In [100]:
population = generate_population(4)
for individu in population:
    # print(individu['Timetable'])
    print(individu['Cost'])

eliminate(population)
print("========")

for individu in population:
        # print(individu['Timetable'])
    print(individu['Cost'])


433
420
425
470
420
425


In [86]:
# Mutation

def mutation_func(mutan):

    alpha = 0.001

    for angkatan in mutan:
        for kelas in mutan[angkatan]:
            for hari in mutan[angkatan][kelas]:
                active_func = round(random.uniform(0.0, 1.0), 2)
                alokasi_waktu = 0
                if active_func <= alpha:
                    # print(angkatan)
                    # print(kelas)
                    # print(hari)
                    # print(active_func)
                    mapel_temp = {}

                    list_mapel = deepcopy(test['List_Mapel'])
                    list_guru = deepcopy(test['Guru'])

                    for jam in test['Waktu'][hari]:
                        if (alokasi_waktu == 0):
                            temp_id = {}

                            temp_mapel, index_mapel = randomizer(list_mapel)
                            temp_guru, index_guru = randomizer(list_guru)
                            alokasi_waktu = int(temp_mapel['Alokasi_Jam'])

                            list_mapel.pop(index_mapel)
                            list_guru.pop(index_guru)

                            temp_id['Mapel'] = temp_mapel['Mapel']
                            temp_id['ID'] = temp_mapel['ID']
                            temp_id['Guru'] = temp_guru
                            mapel_temp[temp_mapel['ID']] = temp_id
                            mapel_temp[temp_mapel['ID']]['Slot_waktu'] = []
                        
                        
                        mapel_temp[temp_mapel['ID']]['Slot_waktu'].append(jam)
                        alokasi_waktu = alokasi_waktu - 1
                        
                    mutan[angkatan][kelas][hari] = mapel_temp

In [108]:
# Main Looping
from IPython.display import clear_output
gen_num = 8
population = generate_population(gen_num)
max_loop = 10000

i = 0
while i < max_loop:
    if i % 10 == 0:
        clear_output()
        print(i)
        for individu in population:
            # print(individu['Timetable'])
            print(individu['Cost'])
    parent1, parent2, parent3, parent4, parent5, parent6 = deepcopy(selection(population))
    child1, child2 = deepcopy(crossover(parent1, parent2))
    child3, child4 = deepcopy(crossover(parent3, parent4))
    child5, child6 = deepcopy(crossover(parent5, parent6))
    # print(child1)
    

    mutation_func(child1['Timetable'])
    mutation_func(child2['Timetable'])
    child1['Cost'], guru1 = cost_function(child1['Timetable'])
    child2['Cost'], guru2 = cost_function(child2['Timetable'])
    
    mutation_func(child3['Timetable'])
    mutation_func(child4['Timetable'])
    child3['Cost'], guru1 = cost_function(child3['Timetable'])
    child4['Cost'], guru2 = cost_function(child4['Timetable'])
    
    mutation_func(child5['Timetable'])
    mutation_func(child6['Timetable'])
    child5['Cost'], guru1 = cost_function(child5['Timetable'])
    child6['Cost'], guru2 = cost_function(child6['Timetable'])
    # for j in guru1:
    #     print(j)
    #     for k in guru1[j]['Slot_waktu']:
    #         result = dict((i, guru1[j]['Slot_waktu'][k].count(i)) for i in guru1[j]['Slot_waktu'][k])
            
    #         print(result)
            # cost += len(guru1[j]['Slot_waktu'][k]) - len(result)
    # mutation_func(child2)
    reassign(population, child1, child2)
    reassign(population, child3, child4)
    reassign(population, child5, child6)
    i += 1


In [ ]:
cost

In [ ]:

# Test modifikasi data pada sebuah gen/kromosom/individu

print(space['VII']['A']['Senin']['7:00 - 7:40'])

space['VII']['A']['Senin']['7:00 - 7:40']['Guru'] = 'Yayat'
space['VII']['A']['Senin']['7:00 - 7:40']['Mapel'] = 'PKN'
print(space['VII']['A']['Senin']['7:00 - 7:40'])

# mapel indo
{0, 1, 2}



In [58]:
# Fungsi untuk menulis hasil solusi/gen/kromosom/individu ke file
def write_data(data, path):
    with open(path, 'w') as write_file:
        json.dump(data, write_file, indent=4)

In [59]:
write_data(space, 'output_random.json')


In [ ]:
# Testing only
hasil = []
for i in range(0,2000):
    space = generate_gen()
    hasil.append(cost_function(space))

print(max(hasil))



In [ ]:
# Testing only
for i in range(0,2000):
    space = generate_gen()
    guru = {test['Guru'][i]: {'Mapel':[],'Slot_waktu':[]} for i in range(0, len(test['Guru']))}

    slot_waktu = {}
    for slot_jam in test['Waktu']['Senin']:
        slot_waktu[slot_jam] = []

    # Cost Function
    cost = 0

    for angkatan in space:
        for kelas in space[angkatan]:
            for hari in space[angkatan][kelas]:
                
                mapel_sehari = {}
                mapel_sehari['Mapel'] = []
                mapel_sehari['Total_Jam'] = []
                mapel_sehari['Mapel'].append(
                    space[angkatan][kelas][hari]['7:00 - 7:40']['ID'])
                mapel_sehari['Total_Jam'].append(0)
                counter = 0

                counter_jam = 0
                for jam in space[angkatan][kelas][hari]:
                    # slot_waktu[jam].append(space[angkatan][kelas][hari][jam]['Guru'])
                    
                    # Koneksi guru ke mapel
                    if space[angkatan][kelas][hari][jam]['ID'] not in guru[space[angkatan][kelas][hari][jam]['Guru']]['Mapel']:
                        guru[space[angkatan][kelas][hari][jam]['Guru']]['Mapel'].append(space[angkatan][kelas][hari][jam]['ID'])
                    
                    # print(jam)
                    guru[space[angkatan][kelas][hari][jam]['Guru']]['Slot_waktu'].append(jam)
                    
                    # Mapel sehari ada apa aja ngitung total jam mapel
                    if mapel_sehari['Mapel'][counter] != space[angkatan][kelas][hari][jam]['ID']:
                        mapel_sehari['Mapel'].append(space[angkatan][kelas][hari][jam]['ID'])
                        mapel_sehari['Total_Jam'].append(1)
                        counter += 1
                    else:
                        mapel_sehari['Total_Jam'][counter] += 1
                        continue
                    
                # Jika ada mapel hanya satu jam pada satu hari (+1)
                # print(guru)
                # print(mapel_sehari)
                if 1 in mapel_sehari['Total_Jam']:
                    cost += 1

                # Jika ada Mapel yang sama di satu hari (+1)
                result = dict((i, mapel_sehari['Mapel'].count(i)) for i in mapel_sehari['Mapel'])
                # print(result)
                if len(result) < len(mapel_sehari['Mapel']):
                    cost += 1

    # Jika Guru Memegang lebih dari 4 mapel (+1)
    # print(guru)
    # print(guru['Sakura Pertiwi']['Slot_waktu'])
    for j in guru:
        # print(j)
        # print(guru[j]['Slot_waktu'].count('7:00 - 7:40'))
        result = dict((i, guru[j]['Slot_waktu'].count(i)) for i in guru[j]['Slot_waktu'])
        # print(result)
        cost += len(guru[j]['Slot_waktu']) - len(result)
        # if len(result) < len(guru[j]['Slot_waktu']):
        #     cost +


        if len(guru[j]['Mapel']) > 4:
            cost += 1
        # print(guru[i])

    # Jika ada Guru yang sama di slot waktu yang sama disetiap hari (irisan) (+1)
    # for j in slot_waktu:
    #     result = dict((i, slot_waktu[j].count(i)) for i in slot_waktu[j])
    #     # print(result)
    #     if len(result) < len(slot_waktu[j]):
    #         cost += 1
    #         # print("test")
    # Jika ada mapel yang sama di satu angkatan tapi beda guru (+1)

    print(cost)
